In [0]:
# Este código se ejecuta en una celda de un Notebook de Databricks
# Su responsabilidad es crear una tabla de dimensiones de productos enriquecida en la Capa de Oro (Gold).

from pyspark.sql.functions import col

# 1. --- Configuración de Tablas ---
# Tablas de entrada (ya limpias y estandarizadas)
category_dim_table = "workspace.tecnomundo_data_dimensions.category"
stock_silver_table = "workspace.tecnomundo_data_processed.stock_cleaned"

# Tabla de salida en la capa de Oro
output_gold_table = "workspace.tecnomundo_data_gold.dim_products"

print(f"Tabla de Dimensiones de Categoría: {category_dim_table}")
print(f"Tabla de Stock (Plata): {stock_silver_table}")
print(f"Tabla de Destino (Oro): {output_gold_table}")


# 2. --- Lectura de las Tablas de Entrada ---
print("\nLeyendo tablas de entrada...")
try:
    df_categories = spark.table(category_dim_table)
    df_stock = spark.table(stock_silver_table)
    
    print(f"Se leyeron {df_categories.count()} registros de la tabla de categorías.")
    print("Esquema de la tabla de categorías (verificar nombre de columna aquí):")
    df_categories.printSchema()
    
    print(f"\nSe leyeron {df_stock.count()} registros de la tabla de stock.")
    print("Esquema de la tabla de stock:")
    df_stock.printSchema()

except Exception as e:
    print(f"Error al leer las tablas de entrada: {e}")
    dbutils.notebook.exit("No se pudieron leer las tablas de origen.")


# 3. --- Enriquecimiento de la Dimensión de Productos ---
print("\nEnriqueciendo la dimensión de categorías con la información de stock...")

# Hacemos un LEFT JOIN desde las categorías hacia el stock.
# CORRECCIÓN: La condición del join debe ser explícita para evitar la ambigüedad.
df_enriched_products = df_categories.join(
    df_stock,
    df_categories["codigo_producto"] == df_stock["codigo_producto"],
    "left"
).select(
    # Seleccionamos explícitamente las columnas de la tabla de categorías
    df_categories["codigo_producto"],
    # CORRECCIÓN FINAL: Usamos el nombre de columna correcto 'nombre_del_producto' del esquema.
    df_categories["nombre_del_producto"],
    df_categories["categoria"],
    # Y la columna de stock de la tabla de stock
    df_stock["stock"].alias("stock_actual")
)

print("Join completado.")


# 4. --- Guardado en la Capa de Oro ---
print(f"\nGuardando la dimensión de productos enriquecida en '{output_gold_table}'...")

(df_enriched_products.write
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable(output_gold_table))

print("¡Tabla de dimensión de productos guardada exitosamente en la Capa de Oro!")


# 5. --- Verificación Final ---
print("\nMostrando una muestra de la nueva tabla de dimensiones enriquecida:")
display(spark.table(output_gold_table))

print("\nEsquema final de la tabla de dimensiones:")
spark.table(output_gold_table).printSchema()
